# Portfolio Optimization and Backtesting Analysis

This notebook provides comprehensive portfolio optimization and backtesting capabilities using Modern Portfolio Theory. It includes:

- Interactive Efficient Frontier generation and portfolio selection
- Portfolio optimization using forecasting results
- Comprehensive backtesting workflow with performance analysis
- Strategy vs benchmark comparison and reporting

## Table of Contents
1. [Setup and Data Loading](#setup)
2. [Portfolio Optimization](#optimization)
3. [Portfolio Recommendations](#recommendations)
4. [Backtesting Analysis](#backtesting)
5. [Performance Comparison](#performance)
6. [Final Report](#report)

## 1. Setup and Data Loading {#setup}

In [ ]:
# Import libraries
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json

# Portfolio forecasting modules
from src.data.yfinance_client import YFinanceClient

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Configuration
ASSETS = ['TSLA', 'SPY', 'BND']
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'
BACKTEST_START = '2024-08-01'
BACKTEST_END = '2025-07-31'
INITIAL_CAPITAL = 100000.0

print(f"📊 Assets: {ASSETS}")
print(f"📅 Data period: {START_DATE} to {END_DATE}")
print(f"🔄 Backtest period: {BACKTEST_START} to {BACKTEST_END}")
print(f"💰 Initial capital: ${INITIAL_CAPITAL:,.2f}")

In [ ]:
# Load market data
print("📥 Loading market data...")

client = YFinanceClient()
price_data = {}

for asset in ASSETS:
    try:
        data = client.fetch_data(asset, START_DATE, END_DATE)
        price_data[asset] = data['Close']
        print(f"✅ {asset}: {len(data)} observations loaded")
    except Exception as e:
        print(f"❌ Error loading {asset}: {str(e)}")

# Create combined DataFrame
df = pd.DataFrame(price_data).dropna()
print(f"\n📈 Combined dataset: {len(df)} observations")
display(df.tail())

In [ ]:
# Simple portfolio optimization
print("🚀 Portfolio Optimization")

# Calculate returns
returns = df.pct_change().dropna()

# Calculate expected returns and covariance
expected_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

print("\n📊 Expected Annual Returns:")
for asset in ASSETS:
    print(f"  {asset}: {expected_returns[asset]:.2%}")

print("\n📈 Correlation Matrix:")
corr_matrix = returns.corr()
display(corr_matrix.round(3))

# Simple equal weight portfolio
equal_weights = np.array([1/len(ASSETS)] * len(ASSETS))
portfolio_return = np.sum(expected_returns * equal_weights)
portfolio_vol = np.sqrt(np.dot(equal_weights.T, np.dot(cov_matrix, equal_weights)))
sharpe_ratio = (portfolio_return - 0.02) / portfolio_vol

print(f"\n💼 Equal Weight Portfolio:")
print(f"  Expected Return: {portfolio_return:.2%}")
print(f"  Volatility: {portfolio_vol:.2%}")
print(f"  Sharpe Ratio: {sharpe_ratio:.3f}")

for i, asset in enumerate(ASSETS):
    print(f"  {asset}: {equal_weights[i]:.1%}")

In [ ]:
# Simple backtesting
print("📊 Simple Backtesting Analysis")

# Get backtest period data
backtest_data = df[BACKTEST_START:BACKTEST_END] if BACKTEST_START in df.index else df.tail(252)
backtest_returns = backtest_data.pct_change().dropna()

# Portfolio performance
portfolio_returns = (backtest_returns * equal_weights).sum(axis=1)
cumulative_returns = (1 + portfolio_returns).cumprod()

# Benchmark (SPY)
benchmark_returns = backtest_returns['SPY']
benchmark_cumulative = (1 + benchmark_returns).cumprod()

# Plot performance
plt.figure(figsize=(12, 6))
plt.plot(cumulative_returns.index, cumulative_returns, label='Portfolio', linewidth=2)
plt.plot(benchmark_cumulative.index, benchmark_cumulative, label='SPY Benchmark', linewidth=2)
plt.title('Portfolio vs Benchmark Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Performance metrics
total_return_portfolio = cumulative_returns.iloc[-1] - 1
total_return_benchmark = benchmark_cumulative.iloc[-1] - 1
portfolio_vol_backtest = portfolio_returns.std() * np.sqrt(252)
benchmark_vol_backtest = benchmark_returns.std() * np.sqrt(252)

print(f"\n📈 Backtest Results:")
print(f"Portfolio Total Return: {total_return_portfolio:.2%}")
print(f"Benchmark Total Return: {total_return_benchmark:.2%}")
print(f"Outperformance: {total_return_portfolio - total_return_benchmark:.2%}")
print(f"Portfolio Volatility: {portfolio_vol_backtest:.2%}")
print(f"Benchmark Volatility: {benchmark_vol_backtest:.2%}")

print("\n✅ Portfolio optimization and backtesting completed!")